# DeepState - Experiments

- Using DeepState with gluonts to run some experiments
- Follow this installation guide: http://gluon-ts-staging.s3-accelerate.dualstack.amazonaws.com/PR-271/4/install.html
- Important note! DeepState, DF-RNN are only released for versions "0.4.*"
- For development version, follow **Install from Source Code** (which I did) **Install from Github**

### Deep State Space Models (Rangapuram et al., 2018)

**DeepState** combines state space models (SSM) with a recurrent neural network (RNN). The SSM is applied *locally* to the individual time series that is parametrized using a *global* representation in form of a RNN. The RNN is trained on the entire dataset. 

- [`DeepStateEstimator`](http://gluon-ts.mxnet.io/master/api/gluonts/gluonts.model.deepstate.html#gluonts.model.deepstate.DeepStateEstimator)


In [19]:
!pip install mxnet=="1.4.1"

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
!pip install gluonts

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [59]:
import gluonts
gluonts.__version__

'0.4.1'

In [60]:
import mxnet
mxnet.__version__

'1.4.1'

In [88]:
# if version does not include DeepState, uncomment the following, install development version:
# !pip install git+https://github.com/awslabs/gluon-ts.git

import pprint

# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports 
from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

In [98]:
def deep_state(seed=42, data="m4_quarterly", epochs=100, batches=50):
    
    mx.random.seed(seed)
    np.random.seed(seed)
    
    dataset = get_dataset(data)

    trainer = Trainer(
        ctx=mx.cpu(0),
#         ctx=mx.gpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepStateEstimator(
        trainer=trainer,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,

    )

    predictor = estimator.train(dataset.train)
    
#     predictor = estimator.train(training_data=dataset.train, 
#                                 validation_data=dataset.test)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    
    df=pd.DataFrame([output])
    pprint(output)
    
    return(df)

In [ ]:
%%time
if __name__ == "__main__":
    df = deep_state(seed=42, data="m4_quarterly", epochs=250, batches=100)
df

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.15it/s, avg_epoch_loss=0.127]
INFO:root:Epoch[0] Elapsed time 4.322 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=0.127292
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.62it/s, avg_epoch_loss=0.196]
INFO:root:Epoch[1] Elapsed time 4.236 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=0.196036
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 24.00it/s, avg_epoch_loss=0.096]
INFO:root:Epoch[2] Elapsed time 4.171 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=0.096002
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.50it/s, avg_epoch_loss=-.0252]
INFO:root:Epoch[3] Elapsed time 4.257 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_los

INFO:root:Epoch[36] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.58it/s, avg_epoch_loss=-1.35]
INFO:root:Epoch[36] Elapsed time 4.242 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=-1.350564
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.65it/s, avg_epoch_loss=-1.74]
INFO:root:Epoch[37] Elapsed time 4.231 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=-1.740481
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.79it/s, avg_epoch_loss=-1.5]
INFO:root:Epoch[38] Elapsed time 4.206 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=-1.502040
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.94it/s, avg_epoch_loss=-1.67]
INFO:root:Epoch[39] Elapsed time 4.180 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=-1.673884
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.70it/s, avg_epoch_loss=-

INFO:root:Epoch[73] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.55it/s, avg_epoch_loss=-1.58]
INFO:root:Epoch[73] Elapsed time 4.250 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=-1.580959
INFO:root:Epoch[74] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.19it/s, avg_epoch_loss=-1.72]
INFO:root:Epoch[74] Elapsed time 4.315 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=-1.724690
INFO:root:Epoch[75] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.74it/s, avg_epoch_loss=-2]
INFO:root:Epoch[75] Elapsed time 4.215 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=-2.001124
INFO:root:Epoch[76] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.75it/s, avg_epoch_loss=-1.91]
INFO:root:Epoch[76] Elapsed time 4.212 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=-1.913268
INFO:root:Epoch[77] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.87it/s, avg_epoch_loss=-1.

INFO:root:Epoch[145] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 23.86it/s, avg_epoch_loss=-1.75]
INFO:root:Epoch[145] Elapsed time 4.195 seconds
INFO:root:Epoch[145] Evaluation metric 'epoch_loss'=-1.745674
INFO:root:Epoch[146] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 23.87it/s, avg_epoch_loss=-2.04]
INFO:root:Epoch[146] Elapsed time 4.193 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=-2.040203
INFO:root:Epoch[147] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 23.72it/s, avg_epoch_loss=-1.93]
INFO:root:Epoch[147] Elapsed time 4.219 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=-1.926765
INFO:root:Epoch[148] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 23.82it/s, avg_epoch_loss=-1.87]
INFO:root:Epoch[148] Elapsed time 4.202 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=-1.865861
INFO:root:Epoch[149] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00

INFO:root:Epoch[181] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 23.40it/s, avg_epoch_loss=-2.28]
INFO:root:Epoch[181] Elapsed time 4.275 seconds
INFO:root:Epoch[181] Evaluation metric 'epoch_loss'=-2.276063
INFO:root:Epoch[182] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 23.00it/s, avg_epoch_loss=-2.03]
INFO:root:Epoch[182] Elapsed time 4.349 seconds
INFO:root:Epoch[182] Evaluation metric 'epoch_loss'=-2.028378
INFO:root:Loading parameters from best epoch (172)
INFO:root:Epoch[183] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:04<00:00, 23.76it/s, avg_epoch_loss=-1.84]
INFO:root:Epoch[183] Elapsed time 4.210 seconds
INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=-1.842009
INFO:root:Epoch[184] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:04<00:00, 23.50it/s, avg_epoch_loss=-2.22]
INFO:root:Epoch[184] Elapsed time 4.256 seconds
INFO:root:Epoch[184] Evaluation metric 'epoch_loss'=-2.221555
INFO:root:Epoch[185] Learning rat

INFO:root:Epoch[216] Evaluation metric 'epoch_loss'=-1.972991
INFO:root:Epoch[217] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.52it/s, avg_epoch_loss=-2.38]
INFO:root:Epoch[217] Elapsed time 4.256 seconds
INFO:root:Epoch[217] Evaluation metric 'epoch_loss'=-2.379571
INFO:root:Epoch[218] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.34it/s, avg_epoch_loss=-2.18]
INFO:root:Epoch[218] Elapsed time 4.286 seconds
INFO:root:Epoch[218] Evaluation metric 'epoch_loss'=-2.179409
INFO:root:Epoch[219] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.47it/s, avg_epoch_loss=-2.28]
INFO:root:Epoch[219] Elapsed time 4.263 seconds
INFO:root:Epoch[219] Evaluation metric 'epoch_loss'=-2.275176
INFO:root:Epoch[220] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.58it/s, avg_epoch_loss=-1.88]
INFO:root:Epoch[220] Elapsed time 4.244 seconds
INFO:root:Epoch[220] Evaluation metric 'epoch_loss'=-1.879390
INFO:root:Epoch[221] Learning rate

In [ ]:
df

In [ ]:
%%time
if __name__ == "__main__":
    df = deep_state(seed=42, data="m4_quarterly", epochs=2000, batches=100)
df

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 22.93it/s, avg_epoch_loss=0.127]
INFO:root:Epoch[0] Elapsed time 4.365 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=0.127292
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.24it/s, avg_epoch_loss=0.196]
INFO:root:Epoch[1] Elapsed time 4.305 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=0.196036
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.67it/s, avg_epoch_loss=0.096]
INFO:root:Epoch[2] Elapsed time 4.227 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=0.096002
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.39it/s, avg_epoch_loss=-.0252]
INFO:root:Epoch[3] Elapsed time 4.277 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_los

INFO:root:Epoch[36] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.24it/s, avg_epoch_loss=-1.35]
INFO:root:Epoch[36] Elapsed time 4.307 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=-1.350564
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.30it/s, avg_epoch_loss=-1.74]
INFO:root:Epoch[37] Elapsed time 4.294 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=-1.740481
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.40it/s, avg_epoch_loss=-1.5]
INFO:root:Epoch[38] Elapsed time 4.276 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=-1.502040
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.54it/s, avg_epoch_loss=-1.67]
INFO:root:Epoch[39] Elapsed time 4.252 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=-1.673884
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.47it/s, avg_epoch_loss=-

INFO:root:Epoch[73] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.39it/s, avg_epoch_loss=-1.58]
INFO:root:Epoch[73] Elapsed time 4.279 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=-1.580959
INFO:root:Epoch[74] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 20.36it/s, avg_epoch_loss=-1.72]
INFO:root:Epoch[74] Elapsed time 4.916 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=-1.724690
INFO:root:Epoch[75] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.19it/s, avg_epoch_loss=-2]
INFO:root:Epoch[75] Elapsed time 4.314 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=-2.001124
INFO:root:Epoch[76] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 22.80it/s, avg_epoch_loss=-1.91]
INFO:root:Epoch[76] Elapsed time 4.389 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=-1.913268
INFO:root:Epoch[77] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 22.77it/s, avg_epoch_loss=-1.

INFO:root:Epoch[145] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 23.57it/s, avg_epoch_loss=-1.75]
INFO:root:Epoch[145] Elapsed time 4.246 seconds
INFO:root:Epoch[145] Evaluation metric 'epoch_loss'=-1.745674
INFO:root:Epoch[146] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 23.57it/s, avg_epoch_loss=-2.04]
INFO:root:Epoch[146] Elapsed time 4.245 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=-2.040203
INFO:root:Epoch[147] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 23.55it/s, avg_epoch_loss=-1.93]
INFO:root:Epoch[147] Elapsed time 4.250 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=-1.926765
INFO:root:Epoch[148] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 23.52it/s, avg_epoch_loss=-1.87]
INFO:root:Epoch[148] Elapsed time 4.255 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=-1.865861
INFO:root:Epoch[149] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00

INFO:root:Epoch[181] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 23.64it/s, avg_epoch_loss=-2.28]
INFO:root:Epoch[181] Elapsed time 4.233 seconds
INFO:root:Epoch[181] Evaluation metric 'epoch_loss'=-2.276063
INFO:root:Epoch[182] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 23.55it/s, avg_epoch_loss=-2.03]
INFO:root:Epoch[182] Elapsed time 4.250 seconds
INFO:root:Epoch[182] Evaluation metric 'epoch_loss'=-2.028378
INFO:root:Loading parameters from best epoch (172)
INFO:root:Epoch[183] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:04<00:00, 23.49it/s, avg_epoch_loss=-1.84]
INFO:root:Epoch[183] Elapsed time 4.260 seconds
INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=-1.842009
INFO:root:Epoch[184] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:04<00:00, 23.30it/s, avg_epoch_loss=-2.22]
INFO:root:Epoch[184] Elapsed time 4.294 seconds
INFO:root:Epoch[184] Evaluation metric 'epoch_loss'=-2.221555
INFO:root:Epoch[185] Learning rat

...

INFO:root:Epoch[1875] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.46it/s, avg_epoch_loss=-2.72]
INFO:root:Epoch[1875] Elapsed time 4.264 seconds
INFO:root:Epoch[1875] Evaluation metric 'epoch_loss'=-2.716561
INFO:root:Epoch[1876] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.25it/s, avg_epoch_loss=-2.73]
INFO:root:Epoch[1876] Elapsed time 4.306 seconds
INFO:root:Epoch[1876] Evaluation metric 'epoch_loss'=-2.732622
INFO:root:Epoch[1877] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.41it/s, avg_epoch_loss=-2.62]
INFO:root:Epoch[1877] Elapsed time 4.273 seconds
INFO:root:Epoch[1877] Evaluation metric 'epoch_loss'=-2.620305
INFO:root:Epoch[1878] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 22.51it/s, avg_epoch_loss=-2.59]
INFO:root:Epoch[1878] Elapsed time 4.447 seconds
INFO:root:Epoch[1878] Evaluation metric 'epoch_loss'=-2.587856
INFO:root:Epoch[1879] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 

INFO:root:Epoch[1911] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.44it/s, avg_epoch_loss=-2.56]
INFO:root:Epoch[1911] Elapsed time 4.269 seconds
INFO:root:Epoch[1911] Evaluation metric 'epoch_loss'=-2.559001
INFO:root:Epoch[1912] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.51it/s, avg_epoch_loss=-2.78]
INFO:root:Epoch[1912] Elapsed time 4.258 seconds
INFO:root:Epoch[1912] Evaluation metric 'epoch_loss'=-2.776439
INFO:root:Epoch[1913] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.37it/s, avg_epoch_loss=-2.63]
INFO:root:Epoch[1913] Elapsed time 4.283 seconds
INFO:root:Epoch[1913] Evaluation metric 'epoch_loss'=-2.626889
INFO:root:Epoch[1914] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.44it/s, avg_epoch_loss=-2.85]
INFO:root:Epoch[1914] Elapsed time 4.268 seconds
INFO:root:Epoch[1914] Evaluation metric 'epoch_loss'=-2.849622
INFO:root:Epoch[1915] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 

INFO:root:Epoch[1947] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.33it/s, avg_epoch_loss=-2.57]
INFO:root:Epoch[1947] Elapsed time 4.290 seconds
INFO:root:Epoch[1947] Evaluation metric 'epoch_loss'=-2.567104
INFO:root:Epoch[1948] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.42it/s, avg_epoch_loss=-2.51]
INFO:root:Epoch[1948] Elapsed time 4.272 seconds
INFO:root:Epoch[1948] Evaluation metric 'epoch_loss'=-2.507153
INFO:root:Epoch[1949] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.35it/s, avg_epoch_loss=-2.6]
INFO:root:Epoch[1949] Elapsed time 4.284 seconds
INFO:root:Epoch[1949] Evaluation metric 'epoch_loss'=-2.597177
INFO:root:Epoch[1950] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.33it/s, avg_epoch_loss=-2.73]
INFO:root:Epoch[1950] Elapsed time 4.289 seconds
INFO:root:Epoch[1950] Evaluation metric 'epoch_loss'=-2.734564
INFO:root:Epoch[1951] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 2

INFO:root:Epoch[1983] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.61it/s, avg_epoch_loss=-2.61]
INFO:root:Epoch[1983] Elapsed time 4.238 seconds
INFO:root:Epoch[1983] Evaluation metric 'epoch_loss'=-2.613830
INFO:root:Epoch[1984] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.60it/s, avg_epoch_loss=-2.8]
INFO:root:Epoch[1984] Elapsed time 4.240 seconds
INFO:root:Epoch[1984] Evaluation metric 'epoch_loss'=-2.795545
INFO:root:Epoch[1985] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.35it/s, avg_epoch_loss=-2.4]
INFO:root:Epoch[1985] Elapsed time 4.286 seconds
INFO:root:Epoch[1985] Evaluation metric 'epoch_loss'=-2.395005
INFO:root:Epoch[1986] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.50it/s, avg_epoch_loss=-2.58]
INFO:root:Epoch[1986] Elapsed time 4.258 seconds
INFO:root:Epoch[1986] Evaluation metric 'epoch_loss'=-2.577584
INFO:root:Epoch[1987] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23

{'MASE': 1.44851346,
 'MSIS': 39.26617675,
 'epochs': 2000,
 'sMAPE': 0.12405207,
 'seed': 42,
 'wQuantileLoss[0.5]': 0.11575022,
 'wQuantileLoss[0.9]': 0.09880469}
CPU times: user 4h 55min 39s, sys: 32min 59s, total: 5h 28min 39s
Wall time: 2h 28min 27s


In [ ]:
df

,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],epochs,seed
0,1.448513,0.124052,39.266177,0.11575,0.098805,2000,42


In [ ]:
!jupyter nbconvert --output-dir="./html_outputs" --to html DeepState01-DeepState_experiments_with_gluonts.ipynb

[NbConvertApp] Converting notebook DeepState01-DeepState_experiments_with_gluonts.ipynb to html
[NbConvertApp] Writing 1312413 bytes to ./html_outputs/DeepState01-DeepState_experiments_with_gluonts.html


In [7]:
# %%time
# results = []

# if __name__ == "__main__":
#     for i in range(42, 45):
#         print("Seed:", i)
#         res = deep_state(data="m4_weekly", seed=i, epochs=1, batches=25)
#         pprint(res)
#         results.append(res)
    
# results